In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO("./best.pt")

def detect_image(image):
    model = YOLO("./best.pt")  # 모델 로드
    results = model.predict(image, imgsz=1280, save=False)  # YOLO 예측 실행
    
    result_image = results[0].plot()  # 예측 결과 이미지 변환
    return result_image  # 결과 반환

# Gradio UI 설정
demo = gr.Interface(
    fn=detect_image,  # 이미지 분석 함수
    inputs=gr.Image(type="numpy"),  # 입력: 이미지 업로드
    outputs=gr.Image(type="numpy"),  # 출력: 분석된 이미지
    title="YOLO Image Detection",  # UI 제목
    description="Upload an image to detect objects using YOLO model."
)

demo.launch(share=True)  # UI 실행

In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
import tempfile
import os
import ffmpeg

# YOLO 모델 로드
model = YOLO("./best.pt")

def detect_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    # 비디오 정보 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # FPS 보정 (0이면 최소 1로 설정)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fps = max(1, fps)  # 0이면 1로 설정


    # 임시 파일 생성
    temp_out_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # macOS에서는 'mp4v', Windows는 'XVID' 가능
    out = cv2.VideoWriter(temp_out_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # YOLO 모델 예측 --> cuda 사용가능하면 device="cuda"로 변경, cpu는 device 항목 제거. 현재는 MAC의 mps 사용
        results = model.predict(frame, imgsz=1280, save=False, device="mps")
        result_frame = results[0].plot()
        
        # 결과 프레임 저장 (float 형식일 경우 uint8로 변환)
        if result_frame.dtype != np.uint8:
            result_frame = (result_frame * 255).astype(np.uint8)
        
        out.write(result_frame)

    cap.release()
    out.release()
    
    return temp_out_path  # 감지된 비디오 경로 반환

# Gradio UI 설정
demo = gr.Interface(
    fn=detect_video,
    inputs=gr.Video(format="mp4"),  # 'type' 대신 'format' 사용
    outputs=gr.Video(format="mp4"),  # mp4 형식 지정
    title="YOLO Video Detection",
    description="Upload a video to detect objects using YOLO model."
)

demo.launch(share=True)


In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
import tempfile
import os
import torch
import subprocess
import ffmpeg

# YOLO 모델 로드 (GPU 또는 MPS 지원 설정)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
model = YOLO("./best.pt").to(device)

def detect_video(video_path, frame_skip=2):
    cap = cv2.VideoCapture(video_path)

    # 비디오 정보 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # FPS 보정 (0이면 기본값 30fps로 설정)
    fps = fps if fps > 0 else 30

    # OpenCV VideoWriter를 사용하여 임시 AVI 파일 저장
    temp_out_path = tempfile.NamedTemporaryFile(delete=False, suffix=".avi").name
    temp_final_out = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name  # 최종 변환된 MP4 파일

    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 'XVID'는 대부분의 OS에서 호환됨
    out = cv2.VideoWriter(temp_out_path, fourcc, fps, (width, height))

    frame_count = 0
    frames = []
    results_cache = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            frames.append(frame)

            if len(frames) >= 4:
                results = model.predict(frames, imgsz=640, device=device, save=False)
                results_cache.extend(results)
                frames = []

        frame_count += 1

    if frames:
        results = model.predict(frames, imgsz=640, device=device, save=False)
        results_cache.extend(results)

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            result_frame = results_cache.pop(0).plot()
            result_frame = cv2.cvtColor(result_frame, cv2.COLOR_RGB2BGR)

            if result_frame.dtype != np.uint8:
                result_frame = (result_frame * 255).astype(np.uint8)

            out.write(result_frame)
        else:
            out.write(frame)

        frame_count += 1

    cap.release()
    out.release()

    # `subprocess`를 사용하여 ffmpeg로 MP4 변환
    subprocess.run([
        "/Users/k23070952/Library/Python/3.9/lib/python/site-packages/ffmpeg", "-i", temp_out_path, "-vcodec", "libx264", "-crf", "23", temp_final_out, "-y"
    ], check=True)

    # 변환 후 원본 AVI 파일 삭제
    os.remove(temp_out_path)

    return temp_final_out  # 변환된 MP4 파일 반환

# ✅ **Gradio UI 수정 (기본 `gr.Video` 사용)**
demo = gr.Interface(
    fn=detect_video,
    inputs=[
        gr.Video(format="mp4"),  # 동영상 업로드
        gr.Slider(1, 5, value=2, step=1, label="Frame Skip")  # 프레임 스킵 설정
    ],
    outputs=[
        gr.Video(format="mp4"),  # 🚀 Gradio에서 직접 비디오 재생 가능!
        gr.File(label="Download Processed Video")  # 📥 다운로드 링크 제공
    ],
    title="YOLO Video Detection",
    description="Upload a video to detect objects using YOLO model. Adjust frame skip to speed up processing."
)

demo.launch(share=True)


In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
import tempfile
import os
import ffmpeg

# YOLO 모델 로드
model = YOLO("./best.pt")

def detect_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    # 비디오 정보 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # FPS 보정 (0이면 최소 1로 설정)
    fps = max(1, fps)  

    # 임시 원본 출력 비디오 (mp4v 코덱)
    temp_out_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter(temp_out_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # YOLO 모델 예측
        results = model.predict(frame, imgsz=1280, save=False, device="mps")
        result_frame = results[0].plot()
        
        # 결과 프레임 저장 (float 형식일 경우 uint8로 변환)
        if result_frame.dtype != np.uint8:
            result_frame = (result_frame * 255).astype(np.uint8)
        
        out.write(result_frame)

    cap.release()
    out.release()
    
    # ✅ 비디오를 H.264 (AVC) 코덱으로 변환 (브라우저 호환)
    temp_h264_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name
    ffmpeg.input(temp_out_path).output(temp_h264_path, vcodec="libx264", crf=23).run(overwrite_output=True)
    
    # 원본 임시 파일 삭제
    os.remove(temp_out_path)

    return temp_h264_path  # 변환된 비디오 경로 반환

# Gradio UI 설정
demo = gr.Interface(
    fn=detect_video,
    inputs=gr.Video(format="mp4"),
    outputs=gr.Video(format="mp4"),  
    title="YOLO Video Detection",
    description="Upload a video to detect objects using YOLO model."
)

demo.launch()


Running on local URL:  http://127.0.0.1:7886


In [49]:
!pip uninstall ffmpeg -y
!pip install ffmpeg-python


Found existing installation: ffmpeg 1.4
Uninstalling ffmpeg-1.4:
  Successfully uninstalled ffmpeg-1.4
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [51]:
!ffmpeg -version


zsh:1: command not found: ffmpeg
